In [6]:
import csv
import re

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name,
    encoding='utf-8'
):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    # Open the CSV file
    with open(csv_file, newline='', encoding=encoding, errors='ignore') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "FLOAT" or data_type == "INTEGER":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")
                        # Handle other data types as needed

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w', encoding=encoding, errors='ignore') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [7]:
create_stream_statement = """
CREATE TABLE cpd_traffic_crashes (
	crash_record_id INTEGER,
	crash_date_est_i VARCHAR,
	crash_date VARCHAR,
	posted_speed_limit INTEGER,
	traffic_control_device VARCHAR,
	device_condition VARCHAR,
	weather_condition VARCHAR,
	lighting_condition VARCHAR,
	first_crash_type VARCHAR,
	trafficway_type VARCHAR,
	lane_cnt INTEGER,
	alignment VARCHAR,
	roadway_surface_cond VARCHAR,
	road_defect VARCHAR,
	report_type VARCHAR,
	crash_type VARCHAR,
	intersection_related_i VARCHAR,
	not_right_of_way_i VARCHAR,
	hit_and_run_i VARCHAR,
	damage VARCHAR,
	date_police_notified TIMESTAMP,
	prim_contributory_cause VARCHAR,
	sec_contributory_cause VARCHAR,
	street_num INTEGER,
	street_direction VARCHAR,
	street_name VARCHAR,
	beat_of_occurence INTEGER,
	photos_taken_i VARCHAR,
	statements_taken_i VARCHAR,
	dooring_i VARCHAR,
	work_zone_i VARCHAR,
	work_zone_type VARCHAR,
	workers_present_i VARCHAR,
	num_units INTEGER,
	most_severe_injury VARCHAR,
	injuries_total INTEGER,
	injuries_fatal INTEGER,
	injuries_incapacitating INTEGER,
	injuries_non_incapacitating INTEGER,
	injuries_reported_not_evident INTEGER,
	injuries_no_indication INTEGER,
	injuries_unknown INTEGER,
	crash_hour INTEGER,
	crash_day_of_week INTEGER,
	crash_month INTEGER,
	latitude FLOAT,
	longitude FLOAT,
	crash_location VARCHAR 
);
"""

input_data_file = '/Users/briandunn/Desktop/Apache Kafka- Kafka Connect/Chicago Traffic Crashes CPD/data/Traffic_Crashes-Crashes.csv'

output_file_location = '/Users/briandunn/Desktop/Apache Kafka- Kafka Connect/Chicago Traffic Crashes CPD/statements/inserts/cpd_car_crashes_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "cpd_traffic_crashes"
    )